In [1]:
# Read data from text file
with open('/Users/jyotit-kaushal/github/misc-testing/LA_eval_small/ASVspoof2019.LA.cm.eval.trl.txt', 'r') as file:
    lines = file.readlines()

# Convert "- -" to "1" and "-" to "2" in each line
for i in range(len(lines)):
    lines[i] = lines[i].replace('- -', 'XX').replace('-', 'XXXX')

data = [line.strip().split(' ') for line in lines]

# Write data to CSV file
with open('y_init.csv', 'w') as f:
    f.write("la_code,la_e_code,attack,label\n")  # Write header
    for item in data:
        if item[3] == 'bonafide':
            pass
        else:
            item[3] += ' spoof'
        f.write(f"{item[0]},{item[1]},{item[2]}, {item[3]}\n")



In [2]:
import os
import csv

# Directory path
directory = '/Users/jyotit-kaushal/github/misc-testing/LA_eval_small/ASVspoof2019_LA_eval/flac'  # Change this to your directory path

# List all filenames in the directory
filenames = os.listdir(directory)

# Extract file names without extensions
filenames_without_extensions = [os.path.splitext(filename)[0] for filename in filenames]

# Write filenames to a CSV file
with open('X_init.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['la_e_code'])  # Write header
    for filename in filenames_without_extensions:
        writer.writerow([filename])


In [3]:
from random import randrange
import pandas as pd

X= pd.read_csv('X_init.csv')
X = pd.DataFrame(X)

y= pd.read_csv('y_init.csv')
y= pd.DataFrame(y)

X= X.sort_values(by='la_e_code', ascending=True)
y= y.sort_values(by='la_e_code', ascending=True)

# X.head()
# y.head()



/var/folders/47/sx3slfp54xv_8syv33c8kzz80000gn/T/ipykernel_61526/1689657859.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
def subsample(X, y, ratio=1):
    temp_X = list()
    temp_y = list()
    picked_indices= set()

    n_sample = round(len(X) * ratio)

    while len(temp_X) < n_sample:
        index = randrange(len(X))

        if index not in picked_indices:
            temp_X.append(X.iloc[index])
            temp_y.append(y.iloc[index])
            picked_indices.add(index)

    temp_X_df = pd.DataFrame(temp_X)
    temp_y_df = pd.DataFrame(temp_y)

    return temp_X_df, temp_y_df

num_bags = 1
ratio = 0.65

# bags_X = list()
# bags_y= list()

sample_X, sample_y= subsample(X, y, ratio)

# for i in range(num_bags):  # Corrected the range to iterate over all bags
#     sample_X, sample_y = subsample(X, y, ratio)

#     bags_X.append(sample_X)
#     bags_y.append(sample_y)

In [5]:
print(sample_X.head())
print(sample_y.head())

      la_e_code
0  LA_E_5932896
   la_code     la_e_code attack       label
0  LA_0039  LA_E_2834763   XXXX   A11 spoof


In [6]:
import shutil


sample_X['la_e_code'] = sample_X['la_e_code'].apply(lambda x: x + '.flac')

source_directory = '/Users/jyotit-kaushal/github/misc-testing/LA_eval_small/ASVspoof2019_LA_eval/flac'
destination_directory = '/Users/jyotit-kaushal/github/misc-testing/ensembling/data1/flac'

for filename in sample_X['la_e_code']:
    source_file = os.path.join(source_directory, filename)
    destination_file = os.path.join(destination_directory, filename)
    shutil.copy(source_file, destination_file)

print("Files moved successfully.")

Files moved successfully.


In [7]:
import csv

# Read data from CSV file
data = []
with open('output.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append(row)

# Modify data
for entry in data:
    if entry['attack'] == 'XX':
        entry['attack'] = '- -'
    elif entry['attack'] == 'XXXX':
        entry['attack'] = '-'

# Write data to text file
with open('output_text.txt', 'w') as txtfile:
    txtfile.write("la_code,la_e_code,attack,label\n")
    for entry in data:
        txtfile.write(f"{entry['la_code']} {entry['la_e_code']} {entry['attack']}{entry['label']}\n")


In [8]:
# Define input and output file paths
input_file = 'output_text.txt'
output_file = '/Users/jyotit-kaushal/github/misc-testing/ensembling/data1/flac/protocols/protcol_trainwtv.txt'

# Read lines from input file, skipping the first line
with open(input_file, 'r') as f_in:
    lines = f_in.readlines()[1:]

# Write the remaining lines to the output file
with open(output_file, 'w') as f_out:
    f_out.writelines(lines)